#Initilization

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import plotly.express as px
import plotly

# Backend Testing

In [7]:
store_sales = pd.read_csv("mock_kaggle.csv")
if True:
  date = 'data'
  sales = 'venda'
  sales_diff = 'sales_diff'
  LP = 'venda'
  img = 'Predict.jpg'
  drpdata = ['estoque', 'preco']
  pred = 'Prediction'
  predtrue = 'Original'
  predfalse = 'Prediction'
  #Dropping store and item columns
  store_sales = store_sales.drop(drpdata, axis=1)
  #Coverting 'date' string to 'Date' type
  store_sales[date] = pd.to_datetime(store_sales[date])
  #Sumation of sales in months
  store_sales[date] = store_sales[date].dt.to_period("M")
  monthly_sales = store_sales.groupby(date).sum().reset_index()
  #Converting resulting 'date' type to 'timestamp' datatype
  monthly_sales[date] = monthly_sales[date].dt.to_timestamp()
  # Visualization
  #plt.figure(figsize=(15,5))
  #plt.plot(monthly_sales[date], monthly_sales[sales])
  #plt.xlabel("Date")
  #plt.ylabel("Sales")
  #plt.title("RESULT")
  #plt.show()
  # Finding Difference of sales month
  monthly_sales[sales_diff] = monthly_sales[sales].diff()
  monthly_sales = monthly_sales.dropna()
  # Dropping 'sales' and 'date'
  supervised_data = monthly_sales.drop([date, sales], axis=1)
  # Preparing Supervised Data
  for i in range(1,13):
    col_name = 'month_' + str(i)
    supervised_data[col_name] = supervised_data[sales_diff].shift(i)
  supervised_data = supervised_data.dropna().reset_index(drop=True)
  ## Split data into train and test
  train_data = supervised_data[:-12]
  test_data = supervised_data[-12:]
  #print("Train Data Shape: ", train_data.shape)
  #print("Test Data Shape: ", test_data.shape)
  scaler = MinMaxScaler(feature_range=(-1,1))
  scaler.fit(train_data)
  train_data = scaler.transform(train_data)
  test_data = scaler.transform(test_data)
  x_train, y_train = train_data[:,1:], train_data[:,0:1]
  x_test, y_test = test_data[:,1:], test_data[:,0:1]
  y_train = y_train.ravel()
  y_test = y_test.ravel()
  #print("X_train Shape: ", x_train.shape)
  #print("Y_train Shape: ", y_train.shape)
  #print("X_test Shape: ", x_test.shape)
  #print("Y_test Shape: ", y_test.shape)
  # Make Prediction data frame to merge the predicted sales of all trained algs
  sales_dates = monthly_sales[date][-12:].reset_index(drop=True)
  predict_df = pd.DataFrame(sales_dates)
  act_sales = monthly_sales[sales][-13:].to_list()
  #print(act_sales)
  # To create the linear regression model, and predicted output
  lr_model = LinearRegression()
  lr_model.fit(x_train, y_train)
  lr_pre = lr_model.predict(x_test)
  lr_pre = lr_pre.reshape(-1,1)
  ###This is a set matrix - contains the input features of the test data, and also the predicted data output
  lr_pre_test_set = np.concatenate([lr_pre, x_test], axis=1)
  lr_pre_test_set = scaler.inverse_transform(lr_pre_test_set)
  result_list = []
  for index in range(0, len(lr_pre_test_set)):
    result_list.append(lr_pre_test_set[index][0] + act_sales[index])
  lr_pre_series = pd.Series(result_list, name=LP)
  predict_df = predict_df.merge(lr_pre_series, left_index=True, right_index=True)
  for i in predict_df[date]:
    predict_df[pred] = predtrue

  for i in monthly_sales[date]:
    monthly_sales[pred] = predfalse

  ############ This part need to learn, i think ###############
  lr_mse = np.sqrt(mean_squared_error(predict_df[LP], monthly_sales[sales][-12:]))
  lr_mae = mean_absolute_error(predict_df[LP], monthly_sales[sales][-12:])
  lr_r2 = r2_score = (predict_df[LP], monthly_sales[sales][-12:])
  monthly_sales[sales] = monthly_sales[sales][:-12]
  #print("Linear Regression MSE: ", lr_mse)
  #print("Linear Regression MAE: ", lr_mae)
  #print("Linear Regression R2: ", lr_r2)
  #plt.figure(figsize=(15,5))
  ###Actual Sales
  #plt.plot(monthly_sales[date], monthly_sales[sales])
  ###Predicted Sales
  #plt.plot(predict_df[date], predict_df[LP])
  #plt.xlabel("Date")
  #plt.ylabel("Sales")
  #plt.legend(['Actual sales', 'Predicted Sales'])
  #plt.savefig(img)
  result = pd.concat([monthly_sales.drop(sales_diff, axis=1),predict_df]).dropna()## find solution
  fig = px.scatter(result, x=date, y=sales, color=pred, trendline="lowess", trendline_options=dict(frac=0.1))
  fig.show()
  plotly.offline.plot(fig, filename='lifeExp.html')
  result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 1 to 11
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   data        30 non-null     datetime64[ns]
 1   venda       30 non-null     float64       
 2   Prediction  30 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 960.0+ bytes
